In [15]:
#!/usr/bin/env python
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

In [16]:
from __future__ import print_function
import sys
#from argparse import ArgumentParser
import time

from openvino.inference_engine import IENetwork, IEPlugin

In [17]:
def colorize(y, ycrcb):
    y[y>255] = 255
    y[y<0] = 0

    img = np.zeros((y.shape[0], y.shape[1], 3), np.uint8)
    img[:,:,0] = y
    img[:,:,1] = ycrcb[:,:,1]
    img[:,:,2] = ycrcb[:,:,2]
    img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    
    return img

# PSNR measure, from ANR's code
def PSNR(pred, gt):
    f = pred.astype(float)
    g = gt.astype(float)
    e = (f - g).flatten()
    n = len(e)
    rst = 10*np.log10(n/e.dot(e))
    
    return rst

In [18]:
model_xml = "/home/yue/caffe-rc5/examples/SRCNN/SRCNN_iter_11112465.xml"
model_bin = os.path.splitext(model_xml)[0] + ".bin"
prob_threshold = 0.5
labels_map = None

In [19]:
print("Initializing plugin for CPU device...")
plugin = IEPlugin(device="CPU", plugin_dirs=None)

Initializing plugin for CPU device...


In [20]:
print("Adding CPU extenstions...")
plugin.add_cpu_extension("/opt/intel/computer_vision_sdk/deployment_tools/inference_engine/lib/ubuntu_16.04/intel64/libcpu_extension_sse4.so")

Adding CPU extenstions...


In [21]:
print("Reading IR...")
net = IENetwork.from_ir(model=model_xml, weights=model_bin)
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))

Reading IR...


In [22]:
print("Loading IR to the plugin...")
exec_net = plugin.load(network=net, num_requests=2)
# Read and pre-process input image
n, c, h, w = net.inputs[input_blob]

Loading IR to the plugin...


In [23]:

im_raw = cv2.imread('./Test/Set14/face_2.bmp')

im_resize = im_raw.transpose((2, 0, 1))  # Change data layout from HWC to CHW

im_resize = cv2.resize(im_raw, (w, h))
ycrcb = cv2.cvtColor(im_resize, cv2.COLOR_RGB2YCR_CB)

im_resize = ycrcb[:,:,0]
im_resize = im_resize.reshape((im_resize.shape[0], im_resize.shape[1], 1))

im_input = im_resize

im_input = im_input.reshape((n, c, h, w))



In [24]:
infer_start = time.time()
res = exec_net.infer(inputs={input_blob: im_input})
print('Inference Time Spent: ', (time.time() - infer_start)*1000, 'ms' )

Inference Time Spent:  53.05647850036621 ms


In [25]:
mat = res['conv3'];
mat = np.squeeze(mat, axis=0)


In [26]:
ycrcb = ycrcb[6:-6,6:-6,:]


In [27]:
im_pred = colorize(mat[0,:,:], ycrcb)
im_pred = cv2.resize(im_pred, (im_raw.shape[1],im_raw.shape[0]))

In [28]:
cv2.imshow('prediction',im_pred)
cv2.imshow('input', im_raw)
cv2.waitKey(0)
cv2.destroyAllWindows()